In [40]:
# Required Libraries
import pandas as pd
import numpy as np
import pm4py
import sklearn
import datetime
import time
import pydot as pydot
from sklearn.tree import export_graphviz
from datetime import timedelta
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [2]:
# reading in the xes file, make sure it is located in the directory
log = pm4py.read_xes(r"C:\Users\Adaml\OneDrive\Desktop\BPI Challenge 2017.xes") 
df = pm4py.convert_to_dataframe(log) 
df['time:timestamp']= pd.to_datetime(df['time:timestamp']).dt.tz_convert(None) # Removing the UTC from timestamp

parsing log, completed traces :: 100%|██████████| 31509/31509 [02:42<00:00, 194.49it/s]


In [3]:
#TimeStamp to int

df['end_time'] = df['time:timestamp'].shift(-1)
df['time_diff'] = df['end_time'] - df['time:timestamp']
pre = df
pre['DDay'] = df['time_diff'].dt.components['days']
pre['DHour'] = df['time_diff'].dt.components['hours']
pre['DMinute'] = df['time_diff'].dt.components['minutes']
pre['DSecond'] = df['time_diff'].dt.components['seconds']
pre['DmilSecond'] = df['time_diff'].dt.components['milliseconds']

pre = pre.drop('time:timestamp', axis=1)
pre = pre.drop('end_time', axis = 1)
pre = pre.drop('time_diff', axis = 1)
pre = pre.drop('org:resource', axis = 1)
pre = pre.drop('EventID', axis = 1)
pre = pre.drop('CreditScore', axis = 1)
pre = pre.drop('OfferedAmount', axis = 1)
pre = pre.drop('OfferID', axis = 1)
pre = pre.drop('FirstWithdrawalAmount', axis = 1)
pre = pre.drop('NumberOfTerms', axis = 1)
pre = pre.drop('Accepted', axis = 1)
pre = pre.drop('MonthlyCost', axis = 1)
pre = pre.drop('Selected', axis = 1)
pre = pre.dropna()

pre['DDay'] = pre['DDay'].astype('int64')
pre['DHour'] = pre['DHour'].astype('int64')
pre['DMinute'] = pre['DMinute'].astype('int64')
pre['DSecond'] = pre['DSecond'].astype('int64')
pre['DmilSecond'] = pre['DmilSecond'].astype('int64')

In [12]:
# Changing all variables so they have a numeric representation
# Preparation for hot Encoding

events = pre['concept:name'].factorize()
pre['concept:name'] = events[0]

EventOrigin = pre['EventOrigin'].factorize()
pre['EventOrigin'] = EventOrigin[0]

LoanGoal = pre['case:LoanGoal'].factorize()
pre['case:LoanGoal'] = LoanGoal[0]

Action = pre['Action'].factorize()
pre['Action'] = Action[0]

appType = pre['case:ApplicationType'].factorize()
pre['case:ApplicationType'] = appType[0]

appType = pre['lifecycle:transition'].factorize()
pre['lifecycle:transition'] = appType[0]

pre['next_event'] = pre['concept:name'].shift(-1)
pre = pre.dropna()
pre['next_event'] = pre['next_event'].astype('int64')
pre['case:RequestedAmount'] = pre['case:RequestedAmount'].astype('int64')

pre = pre.drop('case:concept:name', axis = 1)

pre

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

,Action,concept:name,EventOrigin,lifecycle:transition,case:LoanGoal,case:ApplicationType,case:RequestedAmount,DDay,DHour,DMinute,DSecond,DmilSecond,next_event
0,0,0,0,0,0,0,20000,0,0,0,0,48,1
1,1,1,0,0,0,0,20000,0,0,0,0,422,2
2,0,2,1,1,0,0,20000,0,0,1,20,618,2
3,2,2,1,2,0,0,20000,0,0,0,0,11,3
4,0,3,1,1,0,0,20000,0,0,0,0,10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202256,1,8,2,0,1,0,20000,0,0,0,0,12,3
1202257,2,3,1,0,1,0,20000,0,0,0,0,6,9
1202258,0,9,1,1,1,0,20000,0,0,0,0,1,9
1202259,3,9,1,3,1,0,20000,0,0,0,0,2,10


In [31]:
# Getting hot encoding because, using 50000 because it overloads the system otherwise
# train = df.loc[:10000]
# hot = pd.get_dummies(train)
# labels = pd.DataFrame()
# labels['event'] = hot['concept:name']
# labels['next_event'] = hot['next_event']
# hot= hot.drop('concept:name', axis = 1) #To seperate Features from Labels
# hot= hot.drop('next_event', axis = 1) 
# Labels are the values we want to predict, I need to learn how to seperate these properly
# Features are the values we want to use to predict
# features = hot.values
# features

from sklearn import preprocessing
enc = preprocessing.OneHotEncoder()
pre = pre.loc[:1000]
enc.fit(pre)
labels = enc.transform(pre).toarray()
feature_list = list(pre.columns)

In [26]:
# Training and Testing Sets
train_features, test_features, train_labels, test_labels = train_test_split(pre, labels, test_size = 0.2,
                                                                           random_state = 42)

In [27]:
# Instantiate model 
rf = RandomForestRegressor(n_estimators= 1000, random_state=42)
# Train the model on training data
rf.fit(train_features, train_labels)
rf_new = RandomForestRegressor(n_estimators = 100, criterion = 'mse', max_depth = None, 
                               min_samples_split = 2, min_samples_leaf = 1)

predictions = rf.predict(test_features)

array([[0.   , 1.   , 0.   , ..., 0.788, 0.   , 0.   ],
       [0.   , 0.   , 1.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.046, 0.   ],
       ...,
       [0.   , 1.   , 0.   , ..., 0.123, 0.   , 0.   ],
       [0.   , 1.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.008, 0.   ]])

In [28]:
labels = enc.inverse_transform(predictions)
dfs = pd.DataFrame(labels)
dfs

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,20,2,0,0,0,13000,0,0,0,0,8,20
1,2,9,1,5,2,0,12000,-58,2,32,4,743,0
2,4,9,1,4,2,0,5000,4,1,41,56,744,9
3,4,9,1,4,2,0,5000,1,0,18,49,859,6
4,1,13,2,0,2,0,5000,0,0,2,58,595,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,1,1,0,0,2,0,12000,0,0,0,0,201,2
197,0,6,2,0,1,0,10000,0,0,0,1,369,7
198,1,20,2,0,2,0,7500,0,0,0,0,10,9
199,1,5,0,0,1,0,15000,0,0,2,32,592,6


In [41]:
data = series_to_supervised(values, n_in=6)

NameError: name 'series_to_supervised' is not defined

In [39]:
# Import tools needed for visualization

# Pull out one tree from the forest
tree = rf.estimators_[5]

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
graphs = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
graph.write_svg('tree.svg')

FileNotFoundError: [WinError 2] "dot" not found in path.

In [35]:
graph.write_svg('graph.svg')

AttributeError: module 'pydot' has no attribute 'find_graphviz'